# Ticketsimulation in einer Online-Warteschlange

Unser Code simuliert den Verkauf von Tickets für ein Konzert in einer Online-Warteschlange und berechnet: 
- Die Wahrscheinlichkeiten, dass Tickets noch verfügbar sind 
- Die wichitgsten Cutoff-Positionen 
- und erstellt eine Grafische Darstellung der Ticketverfügbarkeiten

## Imports 
- `numpy` für numerische Berechnungen 
- `pandas` für unsere Tabelle
- `matplotlib` für unsere Plot

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

print ("Programm fängt an durchzulaufen")

Programm fängt an durchzulaufen


## Parameter der Simulation
- `TOTAL_TICKETS`: Anzahl Tickets die im Stadion verfügbar sind
- `QUEUE-SIZE`: Anzahl Personen in der Warteschlange
- `SIMULATIONS`: Anzahl der Monte-Carlo-Simulationen 
- `TICKET_VALUES`&`weights`: Mögliche Ticketmengen pro Person un deren Wahrscheinlichkeiten
- `P_ZERO`: Wahrscheinlichkeiten, dass eine Person keine Tickets kauft

In [ ]:
#Parameter
TOTAL_TICKETS=75_000
QUEUE_SIZE=400_000
SIMULATIONS=3_000
TICKETS_VALUES=[1,2,3,4]

#Wahrscheinlichkeit für 0 Tickets 
P_ZERO=0.05
#5%-Cutoff
CUTOFF_PROB=0.05
rng=np.random.default_rng()

#Ticketverteilung 1-4
#nicht gleichverteilt
#Gewicht mit 1/sqrt(k)
#ergibt Erwartungswert=1.65
ticket_value=np.array([1,2,3,4])
weights=1/np.sqrt(ticket_value)
weights/=weights.sum()

## Einzelne Simulation
Simuliert den Ticketverkauf für **eine Warteschlange**: 
- Rückgabe: Position der letzten Person, die noch Tickets bekommt.
- Jede Person entscheidet zufällig, wie viele Tickets sie kauft (0 bis 4, mit Wahrscheinlichkeiten).
- Die Simualtion endet, sobald alle Tickets verkauft sind.
- Die Funktion gibt die Position der letzten Person zurück, die ein Ticket kaufen konte. 

In [ ]:
#Einzelne Simulation
def run_single_simulation():
    tickets_left=TOTAL_TICKETS
    for person in range(QUEUE_SIZE):
        #entscheidet ob Person Tickets kauft
        if rng.random()<P_ZERO:
            demand=0
        else:
            demand=rng.choice(ticket_value, p=weights)
        tickets_left -=demand
        #Tickets sind aufgebraucht
        if tickets_left<0:
            #letzte Person die noch ein Ticket bekommt
            return person
    return QUEUE_SIZE

## Monte-Carlo-Simulation
Die Funktion `run_single_simulation`simuliert die Warteschlange
- Die Funktion `run_single_simulation`wird `SIMULATIONS`-mal aufgerufen.
- Wir wählen eine Schrittweite von 100, um die Berechnungen übersichtlicher machen zu können.
- Berechnet die **Wahrscheinlichkeit**, dass noch Tickets verfügbar sind, für jede Position (`p_ticket_left`)

In [ ]:
#Monte-Carlo-Simulation
sellout_positions=np.array([run_single_simulation()for _ in range(SIMULATIONS)])


#Wahrscheinlichkeit pro Queue-Position mit Schrittweite von 100
positions=np.arange(0, QUEUE_SIZE +1,100)


#Wahrscheinlichkeit das es noch Tickets gibt
p_ticket_left = np.mean(sellout_positions[:, None]> positions, axis=0)

KeyboardInterrupt: 

## Tabelle: Wahrscheinlichkeiten
- Zeigt die Wahrscheinlichkeiten, dass für bestimmte Warteschlangepositionen noch Tickets vefügbar sind.

In [ ]:
#Tabelle um zu sehen welche Postionen in der Warteschlange welche Wahrscheinlichkeit haben noch Tickets zu bekommen
summary_positions=[5_000,10_000,15_000,20_000,25_000,30_000,35_000,40_000,45_000,50_000,55_000,60_000,65_000,75_000]


#Index eimal berechnen 
summary_indices=[np.abs(positions-pos).argmin() for pos in summary_positions]
table=pd.DataFrame({"Queue-Position":summary_positions, 
                    "P(Ticket verfügbar)": [p_ticket_left[i]for i in summary_indices], 
                    "P(Kein Ticket)": [1-p_ticket_left[i]for i in summary_indices]})
table["P(Ticket verfügbar)"]=table["P(Ticket verfügbar)"].round(3)
table["P(Kein Ticket)"]=table["P(Kein Ticket)"].round(3)

## Hier ist der Fokus auf den kritischen Situationen bei denen es schwierig wird noch Tickets zu bekommen
- Analysiert die Postionen zwischen 30.000 und 41.000, wo Tickets knapp werden.
- `imprtant_rows`zeit die relevantesten Positionen zwischen 30.000 und 40.000. 

In [ ]:
#Nur die wichtigsten Zeilen anzeigen und nicht die ganze Tabelle (vorher schonmal ausgegeben und da war wichtigster bereich zwischen 30 und 40.000)
important_positions = [5_000, 25_000, 30_000, 31_000, 32_000, 33_000, 34_000, 35_000, 36_000, 37_000, 38_000, 39_000, 40_000, 41_000]
important_rows = table[table["Queue-Position"].isin(important_positions)]

## Cutoff-Werte
Zeigt Queue-Positionen für verschiedene Wahrscheinlichkeitslevels, ab wann Tickets wahrscheinlich ausverkauft sind.

In [ ]:
#Cutoff-Bestimmung 
levels = [0.5, 0.25, 0.15, 0.10, 0.05, 0.01]
cutoffs = {f"{int(p*100)}% Chance": positions[np.argmax(p_ticket_left < p)]for p in levels}

## Ergebnisse Ausgeben
Hier sollen erstmal nochmal unsere ganzen Anfangsdaten abgebildet werden damit man nochmal sieht aus welchen Werten sich das Ergebnis bildet. 

In [ ]:
#Ergebnisse Ausgeben
print("\n Simulation abgeschlossen")
print(f"Stadion_Tickets: {TOTAL_TICKETS}")
print(f"Simulationen: {SIMULATIONS}")
print(f"Queue-Größe: {QUEUE_SIZE}")

## Sellout-Statistik
- Zusammenfassung der Position, an denen die letzten Tickets verkauft wurden.
- Minimum, Median, Quantile, Maximum.

In [ ]:
#Sellout-Statistik
print("\nSellout-Postion:")
print(f"Minimum: {sellout_positions.min():,}")
print(f"5%-Quantil: {int(np.quantile(sellout_positions, 0.05)):,}")
print(f"Median: {int(np.median(sellout_positions)):,}")
print(f"95%-Quantil: {int(np.quantile(sellout_positions, 0.95)):,}")
print(f"Maximum: {sellout_positions.max():,}")

## Kritische Queue-Positionen für verschiedene Wahrscheinlichkeiten werden angezeigt.

In [ ]:
print("\nKritische Queue-Positionen:")
for k, v in cutoffs.items():
    print(f"{k:>10} ca. {v:,}")

## Visualisierung
- Zeigt die Wahrscheinlichkeit der Ticketverfügbarkeit im kritischen Bereich (20.000-50.000)
- Horizontale Linien: Markieren verschiedene Wahrscheinlichkeitslevel (50%, 10%, 5%). 
- Verikatle Linien: Zeigen die Cutoff-Positionen (die kritischen Positionen an). 

In [ ]:
#Wahrscheinlichkeiten besser darstellen in Diagramm
ZOOM_MIN = 20_000
ZOOM_MAX = 50_000

mask = (positions >= ZOOM_MIN) & (positions <= ZOOM_MAX)

plt.figure(figsize=(9, 5))
plt.plot(positions[mask], p_ticket_left[mask], linewidth=2)

for p in [0.5, 0.1, 0.05]:
    cutoff = positions[np.argmax(p_ticket_left < p)]
    plt.axhline(p, linestyle="--", alpha=0.6)
    plt.axvline(cutoff, linestyle=":", alpha=0.6)

plt.xlabel("Queue-Position")
plt.ylabel("P(Ticket verfügbar)")
plt.title("Bereich in dem die Ticketverfügbarkeit unwahrscheinlicher wird")
plt.grid(True)
plt.savefig("plot_zoom.png")
plt.show()